# **Red neuronal convolucional para el diagnóstico de nódulos tiroideos según la clasificación EU-TIRADS**

## Por Alejandro Martínez Hernández

### Notebook 3/3

# **Creación de modelos**

PAra la siguiente parte se crearan los siguientes modelos de clasificación simple:
- SVM con crossvalidation y gridsearch para buscar parámetros.
- SVM con preprocesamiento de datos con PCA, crossvalidation y gridsearch para buscar parámetros.
- Forest con crossvalidation y gridsearch para buscar parámetros.
- Forest con preprocesamiento de datos con PCA, crossvalidation y gridsearch para buscar parámetros.

## **SVM**

### **Crossvalidation + GridSearch**


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Listas para almacenar los datos de las imágenes y sus etiquetas
data = []
labels = []

# Cargar las imágenes y las etiquetas desde las carpetas
for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        if image is not None:
            # Convertir imagen a escala de grises y redimensionar
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            # Aplanar la imagen para crear un vector de características
            flattened_image = image.flatten()
            data.append(flattened_image)
            labels.append(category)

# Convertir las listas de datos y etiquetas en arrays de numpy
data = np.array(data)
labels = np.array(labels)

# Parámetros para GridSearch
param_grid = {
    'C': [0.1, 1, 10],  # valores de C
    'gamma': ['scale', 'auto', 0.1, 0.001],  # valores de gamma
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']  # tipos de kernel
}

# Crear el modelo SVM
svm_model = svm.SVC()

# Configuración de GridSearchCV
grid_search = GridSearchCV(svm_model, param_grid, cv=5, verbose=1, scoring='accuracy')

# Ejecutar GridSearch
grid_search.fit(data, labels)

# Mejor modelo y parámetros
print("Mejor modelo:", grid_search.best_estimator_)
print("Mejor conjunto de parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


### **PCA + Crossvalidation + GridSearch**

En este código, se han modificado varias partes del anterior para incluir PCA:

- **Pipeline de sklearn:** Utiliza Pipeline para encadenar PCA y SVM. Esto garantiza que PCA se aplique a los datos antes de que se pase a SVM en cada iteración del proceso de entrenamiento y validación cruzada.

- **Parámetros de GridSearch:** Se actualizó param_grid para ajustarlo al pipeline, especificando que los parámetros se apliquen al estimador SVM dentro del pipeline (notado por el prefijo 'svm__' en los nombres de los parámetros).

#### **Número de Componentes para PCA**
Este será un método que se aplicará en cada uno de los modelos para explorar si mejoran o no la selección.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Función para cargar y procesar imágenes
def load_and_preprocess_images(data_dir, categories, image_size=(128, 128)):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(data_dir, category)
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, image_size)
                flattened_image = image.flatten()
                data.append(flattened_image)
                labels.append(category)
    return np.array(data), np.array(labels)

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Cargar y procesar las imágenes
data, labels = load_and_preprocess_images(data_dir, categories)

# Realizar PCA para determinar el número de componentes principales
pca = PCA()
pca.fit(data)

# Graficar porcentaje de varianza acumulada vs número de componentes principales
explained_variance = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--')
plt.title('Número de componentes principales vs Porcentaje de varianza acumulada')
plt.xlabel('Número de componentes principales')
plt.ylabel('Porcentaje de varianza acumulada')
plt.grid(True)
plt.show()

Se conoce que no hay un número "correcto" universal para el número de componentes; depende del equilibrio entre la simplicidad del modelo y la cantidad de varianza explicada. Para los siguientes modelos se tomara un número de componentes principales igual a 50 con lso que se pueden explicar poco más del 70% de la varianza. 

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Listas para almacenar los datos de las imágenes y sus etiquetas
data = []
labels = []

# Cargar las imágenes y las etiquetas desde las carpetas
for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        if image is not None:
            # Convertir imagen a escala de grises y redimensionar
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            # Aplanar la imagen para crear un vector de características
            flattened_image = image.flatten()
            data.append(flattened_image)
            labels.append(category)

# Convertir las listas de datos y etiquetas en arrays de numpy
data = np.array(data)
labels = np.array(labels)

# Definir el número de componentes para PCA
n_components = 50  # Este valor puede ser ajustado según la varianza a conservar

# Crear un pipeline que incluya PCA y SVM
pipeline = Pipeline([
    ('pca', PCA(n_components=n_components)),
    ('svm', svm.SVC())
])

# Parámetros para GridSearch
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 'auto', 0.1, 0.001],
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Configuración de GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, scoring='accuracy')

# Ejecutar GridSearch
grid_search.fit(data, labels)

# Mejor modelo y parámetros
print("Mejor modelo:", grid_search.best_estimator_)
print("Mejor conjunto de parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


## **Random Forest**

### **Crossvalidation + GridSearch**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Listas para almacenar los datos de las imágenes y sus etiquetas
data = []
labels = []

# Cargar las imágenes y las etiquetas desde las carpetas
for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        if image is not None:
            # Convertir imagen a escala de grises y redimensionar
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            # Aplanar la imagen para crear un vector de características
            flattened_image = image.flatten()
            data.append(flattened_image)
            labels.append(category)

# Convertir las listas de datos y etiquetas en arrays de numpy
data = np.array(data)
labels = np.array(labels)

# Parámetros para GridSearch
param_grid = {
    'n_estimators': [10, 100],  # Número de árboles en el bosque
    'max_features': ['auto', 'sqrt', 'log2'],  # Número de características a considerar al buscar la mejor división
    'max_depth': [None, 10, 20],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5],  # Número mínimo de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2],  # Número mínimo de muestras requeridas en cada hoja del árbol
    'bootstrap': [True, False]  # Método para muestrear puntos de datos (con o sin reemplazo)
}

# Crear el modelo de Random Forest
forest_model = RandomForestClassifier()

# Configuración de GridSearchCV
grid_search = GridSearchCV(forest_model, param_grid, cv=5, verbose=1, scoring='accuracy')

# Ejecutar GridSearch
grid_search.fit(data, labels)

# Mejor modelo y parámetros
print("Mejor modelo:", grid_search.best_estimator_)
print("Mejor conjunto de parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
240 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\base.py", line 6

Mejor modelo: RandomForestClassifier(bootstrap=False, min_samples_split=5, n_estimators=10)
Mejor conjunto de parámetros: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.702659574468085


### **PCA + Crossvalidation + GridSearch**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Listas para almacenar los datos de las imágenes y sus etiquetas
data = []
labels = []

# Cargar las imágenes y las etiquetas desde las carpetas
for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        if image is not None:
            # Convertir imagen a escala de grises y redimensionar
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            # Aplanar la imagen para crear un vector de características
            flattened_image = image.flatten()
            data.append(flattened_image)
            labels.append(category)

# Convertir las listas de datos y etiquetas en arrays de numpy
data = np.array(data)
labels = np.array(labels)

# Definir el número de componentes principales para PCA
n_components = 50

# Crear un pipeline que incluya PCA y Random Forest
pipeline = Pipeline([
    ('pca', PCA(n_components=n_components)),
    ('forest', RandomForestClassifier())
])

# Parámetros para GridSearch
param_grid = {
    'forest__n_estimators': [10, 100],
    'forest__max_features': ['auto', 'sqrt', 'log2'],
    'forest__max_depth': [None, 10, 20],
    'forest__min_samples_split': [2, 5],
    'forest__min_samples_leaf': [1, 2],
    'forest__bootstrap': [True, False]
}

# Configuración de GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, scoring='accuracy')

# Ejecutar GridSearch
grid_search.fit(data, labels)

# Mejor modelo y parámetros
print("Mejor modelo:", grid_search.best_estimator_)
print("Mejor conjunto de parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
240 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\alema\Desktop\pape

Mejor modelo: Pipeline(steps=[('pca', PCA(n_components=150)),
                ('forest',
                 RandomForestClassifier(bootstrap=False, max_depth=20))])
Mejor conjunto de parámetros: {'forest__bootstrap': False, 'forest__max_depth': 20, 'forest__max_features': 'sqrt', 'forest__min_samples_leaf': 1, 'forest__min_samples_split': 2, 'forest__n_estimators': 100}
Mejor puntuación de validación cruzada: 0.6609042553191491


## **Naive Bayes**

### **Crossvalidation + GridSearch**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Listas para almacenar los datos de las imágenes y sus etiquetas
data = []
labels = []

# Cargar las imágenes y las etiquetas desde las carpetas
for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        if image is not None:
            # Convertir imagen a escala de grises y redimensionar
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            # Aplanar la imagen para crear un vector de características
            flattened_image = image.flatten()
            data.append(flattened_image)
            labels.append(category)

# Convertir las listas de datos y etiquetas en arrays de numpy
data = np.array(data)
labels = np.array(labels)

# Crear el modelo de Naive Bayes
nb = GaussianNB()

# Definir los parámetros para GridSearch (Naive Bayes no tiene muchos parámetros ajustables)
param_grid = {
    # No hay parámetros específicos de GaussianNB para ajustar con GridSearch
}

# Configuración de GridSearchCV
grid_search = GridSearchCV(nb, param_grid, cv=5, verbose=1, scoring='accuracy')

# Ejecutar GridSearch
grid_search.fit(data, labels)

# Mejor modelo y parámetros
print("Mejor modelo:", grid_search.best_estimator_)
print("Mejor conjunto de parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Mejor modelo: GaussianNB()
Mejor conjunto de parámetros: {}
Mejor puntuación de validación cruzada: 0.6109042553191489


### **PCA + Crossvalidation + GridSearch**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Ruta relativa a la carpeta que contiene las imágenes organizadas por etiquetas
data_dir = "db_unal/organized/images/cropped"
categories = ['high', 'low']

# Listas para almacenar los datos de las imágenes y sus etiquetas
data = []
labels = []

# Cargar las imágenes y las etiquetas desde las carpetas
for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        if image is not None:
            # Convertir imagen a escala de grises y redimensionar
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            # Aplanar la imagen para crear un vector de características
            flattened_image = image.flatten()
            data.append(flattened_image)
            labels.append(category)

# Convertir las listas de datos y etiquetas en arrays de numpy
data = np.array(data)
labels = np.array(labels)

# Definir el número de componentes principales para PCA
n_components = 50

# Crear un pipeline que incluya PCA y Naive Bayes
pipeline = Pipeline([
    ('pca', PCA(n_components=n_components)),
    ('naive_bayes', GaussianNB())
])

# Parámetros para GridSearch (aunque Naive Bayes no tiene muchos parámetros ajustables)
param_grid = {
    'pca__n_components': [100, 150, 192]  # Ejemplos de posibles valores para n_components
}

# Configuración de GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, scoring='accuracy')

# Ejecutar GridSearch
grid_search.fit(data, labels)

# Mejor modelo y parámetros
print("Mejor modelo:", grid_search.best_estimator_)
print("Mejor conjunto de parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alema\Desktop\paper\nodulos_tiroideos\venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\alema\Desktop\paper\nod

Mejor modelo: Pipeline(steps=[('pca', PCA(n_components=100)), ('naive_bayes', GaussianNB())])
Mejor conjunto de parámetros: {'pca__n_components': 100}
Mejor puntuación de validación cruzada: 0.6526595744680851
